In [ ]:
import pandas as pd
import re
import os
import xml.etree.ElementTree as ET
from datetime import datetime

# === Папки ===
input_root = "wildberries"   # структура: wildberries/Buyer1/*.csv
output_csv = "final_csv"
output_xlsx = "final_xlsx"
output_xml = "final_xml"
output_orphan = "final_orphan"

for folder in [output_csv, output_xlsx, output_xml, output_orphan]:
    os.makedirs(folder, exist_ok=True)

# === Файлы справочников ===
database_file = "database.csv"
buyers_file = "database_buyers.csv"
seller_file = "database_seller.csv"

# === Загрузка справочников ===
# === Файлы справочников ===
database_file = "database.csv"
buyers_file = "database_buyers.csv"
seller_file = "database_seller.csv"

# === Загрузка справочников ===
db = pd.read_csv(database_file, sep=";", dtype=str).fillna("")

# ⚡ фиксим регистр колонок (делаем все строчными)
db.columns = db.columns.str.lower()

buyers = pd.read_csv(buyers_file, sep=";", dtype=str).set_index("BuyerID").fillna("")
seller = pd.read_csv(seller_file, sep=";", dtype=str).iloc[0].to_dict()


def extract_gtin(kiz: str):
    """Извлекаем GTIN (13 цифр после префикса 010)"""
    if not isinstance(kiz, str):
        return None
    clean = kiz.strip().strip('"').strip("'")
    match = re.match(r"010(\d{13})", clean)
    return match.group(1) if match else None


def create_xml(file, buyer, records_df, seller):
    """Формируем XML УПД 5.03"""
    today = datetime.today().strftime("%d.%m.%Y")
    base_name = os.path.splitext(os.path.basename(file))[0]

    # Дата отгрузки = из названия файла
    try:
        ship_date = datetime.strptime(base_name, "%d.%m.%Y").strftime("%d.%m.%Y")
    except:
        ship_date = today

    # === Формируем уникальный ИдФайл ===
    id_file = f"ON_NSCHFDOPPR_{seller['INN']}_{buyer['INN']}_{today.replace('.','')}_0001"

    root = ET.Element("Файл", {
        "ИдФайл": id_file,
        "ВерсФорм": "5.03",
        "ВерсПрог": "KIZProcessor 1.0"
    })

    doc = ET.SubElement(root, "Документ", {
        "КНД": "1115131",
        "Функция": "СЧФДОП",
        "ПоФактХЖ": "Документ об отгрузке товаров",
        "НаимДокОпр": "Счет-фактура и документ об отгрузке",
        "ДатаИнфПр": today,
        "ВремИнфПр": datetime.now().strftime("%H.%M.%S")
    })

    # === СвСчФакт ===
    sf = ET.SubElement(doc, "СвСчФакт", {
        "НомерДок": id_file,  # теперь совпадает с ИдФайл
        "ДатаДок": today
    })

    # Продавец (ИП)
    svprod = ET.SubElement(sf, "СвПрод")
    idseller = ET.SubElement(svprod, "ИдСв")
    svip = ET.SubElement(idseller, "СвИП", {
        "ИННФЛ": seller["INN"],
        "ОГРНИП": seller["OGRNIP"]
    })
    ET.SubElement(svip, "ФИО", {
        "Фамилия": seller["Familiya"],
        "Имя": seller["Imya"],
        "Отчество": seller["Otchestvo"]
    })
    adr = ET.SubElement(svprod, "Адрес")
    ET.SubElement(adr, "АдрРФ", {
        "Индекс": seller["Indeks"],
        "КодРегион": seller["KodRegion"],
        "НаимРегион": seller["NaimRegion"],
        "Город": seller["Gorod"],
        "Улица": seller["Ulitsa"],
        "Дом": seller["Dom"],
        "Кварт": seller["Kvart"]
    })

    # Покупатель
    svpokup = ET.SubElement(sf, "СвПокуп")
    idbuyer = ET.SubElement(svpokup, "ИдСв")
    svip_b = ET.SubElement(idbuyer, "СвИП", {
        "ИННФЛ": buyer["INN"],
        "ОГРНИП": buyer["OGRNIP"]
    })
    ET.SubElement(svip_b, "ФИО", {
        "Фамилия": buyer["Familiya"],
        "Имя": buyer["Imya"],
        "Отчество": buyer["Otchestvo"]
    })
    adr_b = ET.SubElement(svpokup, "Адрес")
    ET.SubElement(adr_b, "АдрРФ", {
        "Индекс": buyer["Indeks"],
        "КодРегион": buyer["KodRegion"],
        "НаимРегион": buyer["NaimRegion"],
        "Город": buyer["Gorod"],
        "Улица": buyer["Ulitsa"],
        "Дом": buyer["Dom"],
        "Кварт": buyer["Kvart"]
    })

    ET.SubElement(sf, "ДенИзм", {"КодОКВ": "643", "НаимОКВ": "Российский рубль"})

    # === ТаблСчФакт ===
    tab = ET.SubElement(doc, "ТаблСчФакт")

    row_num = 1
    total_sum = 0
    for (group, price, okei, tax), subset in records_df.groupby(["group", "price", "okei", "tax"]):
        qty = len(subset)
        sum_ = float(price) * qty
        total_sum += sum_

        st = ET.SubElement(tab, "СведТов", {
            "НомСтр": str(row_num),
            "НаимТов": group,
            "ОКЕИ_Тов": okei,
            "НаимЕдИзм": "Штука",
            "КолТов": str(qty),
            "ЦенаТов": price,
            "СтТовБезНДС": str(sum_),
            "НалСт": tax,
            "СтТовУчНал": str(sum_)
        })

        dop = ET.SubElement(st, "ДопСведТов")
        nom = ET.SubElement(dop, "НомСредИдентТов")
        for kiz in subset["val"]:
            ET.SubElement(nom, "КИЗ").text = kiz

        akc = ET.SubElement(st, "Акциз")
        ET.SubElement(akc, "БезАкциз").text = "без акциза"
        sn = ET.SubElement(st, "СумНал")
        ET.SubElement(sn, "БезНДС").text = "без НДС"

        row_num += 1

    vs = ET.SubElement(tab, "ВсегоОпл", {
        "СтТовБезНДСВсего": str(total_sum),
        "СтТовУчНалВсего": str(total_sum)
    })
    snv = ET.SubElement(vs, "СумНалВсего")
    ET.SubElement(snv, "БезНДС").text = "без НДС"

    # ПродПер
    svpp = ET.SubElement(doc, "СвПродПер")
    ET.SubElement(svpp, "СвПер", {
        "СодОпер": "Товары переданы",
        "ДатаПер": ship_date
    }).append(ET.Element("БезДокОснПер"))

    # Подписант
    pod = ET.SubElement(doc, "Подписант", {
        "Должн": "Индивидуальный Предприниматель",
        "СпосПодтПолном": "1"
    })
    ET.SubElement(pod, "ФИО", {
        "Фамилия": seller["Familiya"],
        "Имя": seller["Imya"],
        "Отчество": seller["Otchestvo"]
    })

    # === Сохраняем с именем = ИдФайл ===
    out_file = os.path.join(output_xml, f"{id_file}.xml")
    tree = ET.ElementTree(root)
    tree.write(out_file, encoding="windows-1251", xml_declaration=True)
    print(f"📄 XML сохранен: {out_file}")


def process_file(file, db, buyer):
    original = pd.read_csv(file, sep=",", dtype=str, header=None)
    raw_kiz_row = original.iloc[0].dropna().tolist()
    kiz_row = [val for val in raw_kiz_row if isinstance(val, str) and val.strip().startswith("010")]

    records = []
    orphans = []
    for val in kiz_row:
        gtin = extract_gtin(val)
        if gtin:
            row = db.loc[db["gtin"] == gtin]
            if not row.empty:
                r = row.iloc[0]
                records.append({
                    "val": val,
                    "gtin": gtin,
                    "group": r["group"],
                    "price": r["price"],
                    "okei": r["okei"],
                    "tax": r["tax"]
                })
            else:
                orphans.append((val, gtin))
        else:
            orphans.append((val, ""))

    if not records:
        print(f"⚠️ Нет сопоставленных КИЗ в {file}")
        return

    records_df = pd.DataFrame(records)

    # Сохраняем XML
    create_xml(file, buyer, records_df, seller)


# === Запуск ===
for buyer_id in buyers.index:
    folder = os.path.join(input_root, buyer_id)
    if not os.path.exists(folder):
        continue
    buyer = buyers.loc[buyer_id]
    for file in os.listdir(folder):
        if file.endswith(".csv"):
            process_file(os.path.join(folder, file), db, buyer)

print("🎯 Обработка завершена")
